In [ ]:
import numpy as np
import pandas as pd

import math
import numpy as np
import pandas as pd
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import tensorflow_probability as tfp
from keras.models import Sequential
from keras.layers import Dense, LSTM

from tensorflow.keras.layers import StringLookup

#ignore warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
#改配色方案
# brewer2mpl.get_map args: set name set type number of colors
import matplotlib as mpl

#mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors)
mpl.rcParams['font.family'] = 'Times New Roman'

plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300

### Data Process

In [ ]:
data=pd.read_csv('data.csv')
item=3

data['time']=pd.to_datetime(data['time'],format = '%Y-%m-%d %H:%M:%S')

turbulence = np.zeros((len(data), 1))

speed_for_turbulence = np.array(data['speed(m/s)_%s'%item])

for i in range(5,len(data)-1):
  tmp = speed_for_turbulence[(i-5):(i+1)]
  turbulence[i][0] = np.std(tmp)/(np.mean(tmp)+0.001)

turbulence[turbulence>1] = 1
data['turbulence_%s'%item]=turbulence

angle=np.array(data['angle(°)_%s'%item])
cos_angle=[np.cos(np.pi*(i/180)) for i in angle]
data['angle(°)_%s'%item]=cos_angle

norm_data_original=data[data['state_%s'%item]==10]
norm_data1=norm_data_original[norm_data_original['active_value_%s'%item]==100]
norm_data2=norm_data_original[norm_data_original['active_value_%s'%item]==0]

norm_data=pd.concat([norm_data1,norm_data2])

ab_data=data[data['state_%s'%item]!=10]
limit_data=pd.concat([norm_data_original,norm_data],axis =0).drop_duplicates(keep=False)
limit_data=pd.concat([limit_data,ab_data])

speed_all=np.array(data['speed(m/s)_%s'%item])
power_all=np.array(data['power(kW)_%s'%item])
temperature_all=np.array(data['temperature(℃)_%s'%item])
angle_all=np.array(data['angle(°)_%s'%item])
turbulence_all=np.array(data['turbulence_%s'%item])


speed=np.array(norm_data['speed(m/s)_%s'%item])
power=np.array(norm_data['power(kW)_%s'%item])
temperature=np.array(norm_data['temperature(℃)_%s'%item])
angle=np.array(norm_data['angle(°)_%s'%item])
turbulence=np.array(norm_data['turbulence_%s'%item])

speed_index=norm_data['speed(m/s)_%s'%item].index 

#generate dataset
speed_data=[[] for i in range(120)]
power_data=[[] for i in range(120)]
tem_data=[[] for i in range(120)]
angle_data=[[] for i in range(120)]
turbulence_data=[[] for i in range(120)]

speed_index=norm_data['speed(m/s)_%s'%item].index
for i in speed_index:
    for j in range(120):
        speed_data[j].append(speed_all[i-j])

power_index=norm_data['power(kW)_%s'%item].index

for i in power_index:
    for j in range(120):
        power_data[j].append(power_all[i-j])
    
tem_index=norm_data['temperature(℃)_%s'%item].index

for i in tem_index:
    for j in range(120):
        tem_data[j].append(temperature_all[i-j])

angle_index=norm_data['angle(°)_%s'%item].index

for i in angle_index:
    for j in range(120):
        angle_data[j].append(angle_all[i-j])

turbulence_index=norm_data['turbulence_%s'%item].index

for i in turbulence_index:
    for j in range(120):
        turbulence_data[j].append(turbulence_all[i-j])
        
df=pd.DataFrame()
for j in range(120):
    df['speed_%s'%j]=speed_data[j]
    
df_power=pd.DataFrame()
for j in range(120):
    df_power['power_%s'%j]=power_data[j]

df_tem=pd.DataFrame()
for j in range(120):
    df_tem['tem_%s'%j]=tem_data[j]

df_angle=pd.DataFrame()
for j in range(120):
    df_angle['angle_%s'%j]=angle_data[j]

df_turbulence=pd.DataFrame()
for j in range(120):
    df_turbulence['turbulence_%s'%j]=turbulence_data[j]

speed_df=pd.DataFrame()
speed_df['speed0']=df['speed_0']
speed_df['power1']=df_power['power_1']
speed_df['tem_0']=df_tem['tem_0']
speed_df['angle_0']=df_angle['angle_0']
speed_df['turbulence_0']=df_turbulence['turbulence_0']

speed_df['speed1']=df['speed_1']
speed_df['power2']=df_power['power_2']
speed_df['tem_1']=df_tem['tem_1']
speed_df['angle_1']=df_angle['angle_1']
speed_df['turbulence_1']=df_turbulence['turbulence_1']

speed_df['speed2']=df['speed_2']
speed_df['power3']=df_power['power_3']
speed_df['tem_2']=df_tem['tem_2']
speed_df['angle_2']=df_angle['angle_2']
speed_df['turbulence_2']=df_turbulence['turbulence_2']

speed_df['speed3']=df['speed_3']
speed_df['power4']=df_power['power_4']
speed_df['tem_3']=df_tem['tem_3']
speed_df['angle_3']=df_angle['angle_3']
speed_df['turbulence_3']=df_turbulence['turbulence_3']

speed_df['turbulence_4']=df_turbulence['turbulence_4']

In [ ]:
speed_df=pd.DataFrame()
power=(power-np.min(power))/(np.max(power)-np.min(power))

for i in range(4):
  speed_df['power_%s'%(i+1)]=df_power['power_%s'%(i+1)]
  speed_df['speed_%s'%i]=df['speed_%s'%i]
  speed_df['angle_%s'%i]=df_angle['angle_%s'%i]
  speed_df['tem_%s'%i]=df_tem['tem_%s'%i]
  speed_df['turbulence_%s'%(i+1)]=df_turbulence['turbulence_%s'%(i+1)]

col_name=speed_df.columns
for item in col_name:
    tmp=np.array(speed_df[item])
    tmp_1=(tmp-np.mean(tmp))/np.std(tmp)
    speed_df[item]=tmp_1
    
speed=np.array(speed_df)

In [ ]:
speed_df_offline=speed_df
power_offline=power
data_offline=data
norm_data_offline=norm_data

In [ ]:
data_filtered = pd.concat([data,norm_data],axis=0).drop_duplicates(keep=False)

In [ ]:
data_filtered

In [ ]:
#norm dataset

plt.figure(figsize = (8,6))

#norm_data_sampled=norm_data.sample(frac=0.3)
sns.scatterplot(data_filtered['speed(m/s)_3'],data_filtered['power(kW)_3'], color = "red",marker='x',s=4,label='Filtered data')
sns.scatterplot(norm_data['speed(m/s)_3'],norm_data['power(kW)_3'], color = "blue",s=3,label='Normal data')

plt.xlabel('Wind Speed(m/s)',fontsize=16)
plt.ylabel('Wind Power(kW)',fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
#plt.title('Monthly Wind Speed distribution_%s'%i, fontsize=15)
#plt.savefig(fname="image_result/norm_inland.svg",format="svg",)
plt.show()
plt.legend()

In [ ]:
## all dataset
data_plot=data_offline.sample(frac=0.1)
sns.scatterplot(data_plot['speed(m/s)_3'],data_plot['power(kW)_3'],color="blue",s=2)
plt.xlabel('Wind Speed(m/s)',fontsize=16)
plt.ylabel('Wind Power(kW)',fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
#plt.title('Monthly Wind Speed distribution_%s'%i, fontsize=15)
plt.savefig(fname="image_result/power_offshore.svg",format="svg",)
plt.show()

### Temporal Effect Analysis

Wind Speed

In [ ]:
total_speed=np.arange(5,13,0.1)

speed=np.array(data_offline['speed(m/s)_%s'%item])
power=np.array(data_offline['power(kW)_%s'%item])
power=(power-np.min(power))/(np.max(power)-np.min(power))

for tmp_speed in total_speed:

  speed_index=norm_data_offline[(norm_data_offline['speed(m/s)_%s'%item]>=tmp_speed)&(norm_data_offline['speed(m/s)_%s'%item]<=tmp_speed+0.1)].index
  speed_index_pre=[]
  for i in speed_index:
      speed_index_pre.append(i-24)

  speed_10=speed[speed_index]
  power_10=power[speed_index]

  power_10_small=[]
  power_10_large=[]

  for i in range(len(speed_index)):
      if speed[speed_index_pre[i]]<=tmp_speed:
          power_10_small.append(power[speed_index[i]])
      else:
          power_10_large.append(power[speed_index[i]])
  
  fig, axes = plt.subplots(figsize=(5, 3), dpi=300)

  sns.kdeplot(power_10_small, label=r'Small $V_{t-1}$', fill=True,)
  sns.kdeplot(power_10_large, label=r'Large $V_{t-1}$', fill=True,)
  axes.axvline(np.mean(power_10_small), label='Small mean', color='b', linewidth=2,  linestyle='--')
  axes.axvline(np.mean(power_10_large), label='Large mean', color='orange', linewidth=2, linestyle='--')
  axes.legend(prop={'size': 12},loc='best')
  #probplot(df[target], plot=axes[1])

  axes.tick_params(axis='x', labelsize=16)
  axes.tick_params(axis='y', labelsize=16)
  axes.set_xlabel('Standardized power',fontsize=16)
  axes.set_ylabel('Count',fontsize=16)
  axes.set_title('Speed Range:[%s,%s]'%(round(tmp_speed,1),round(tmp_speed+0.1,1)), fontsize=16, pad=12)
  plt.savefig('image_result/offline/speed_%s.png'%round(tmp_speed,1))

In [ ]:
total_speed=np.arange(5,13,0.1)

speed=np.array(data_inline['speed(m/s)_%s'%item])
power=np.array(data_inline['power(kW)_%s'%item])

for tmp_speed in total_speed:

  speed_index=norm_data_inline[(norm_data_inline['speed(m/s)_%s'%item]>=tmp_speed)&(norm_data_inline['speed(m/s)_%s'%item]<=tmp_speed+0.1)].index
  speed_index_pre=[]
  for i in speed_index:
      speed_index_pre.append(i-24)

  speed_10=speed[speed_index]
  power_10=power[speed_index]

  power_10_small=[]
  power_10_large=[]

  for i in range(len(speed_index)):
      if speed[speed_index_pre[i]]<=tmp_speed:
          power_10_small.append(power[speed_index[i]])
      else:
          power_10_large.append(power[speed_index[i]])
  
  fig, axes = plt.subplots(figsize=(5, 3), dpi=300)

  sns.kdeplot(power_10_small, label=r'Small $V_{t-1}$', fill=True,)
  sns.kdeplot(power_10_large, label=r'Large $V_{t-1}$', fill=True,)
  axes.axvline(np.mean(power_10_small), label='Small mean', color='b', linewidth=2,  linestyle='--')
  axes.axvline(np.mean(power_10_large), label='Large mean', color='orange', linewidth=2, linestyle='--')
  axes.legend(prop={'size': 12},loc='best')
  #probplot(df[target], plot=axes[1])

  axes.tick_params(axis='x', labelsize=16)
  axes.tick_params(axis='y', labelsize=16)
  axes.set_xlabel('Wind Power(KW)',fontsize=16)
  axes.set_ylabel('Frequency',fontsize=16)
  axes.set_title('Speed Range:[%s,%s]'%(round(tmp_speed,1),round(tmp_speed+0.1,1)), fontsize=16, pad=12)
  plt.savefig('image_result/inline/speed_%s.png'%round(tmp_speed,1))

Temperature

In [ ]:
total_speed=np.arange(5,13,0.1)

speed=np.array(data_offline['speed(m/s)_%s'%item])
power=np.array(data_offline['power(kW)_%s'%item])
temporature=np.array(data['temperature(℃)_%s'%item])
power=(power-np.min(power))/(np.max(power)-np.min(power))

for tmp_speed in total_speed:

  speed_index=norm_data_offline[(norm_data_offline['speed(m/s)_%s'%item]>=tmp_speed)&(norm_data_offline['speed(m/s)_%s'%item]<=tmp_speed+0.1)].index
  
  temporature_list=temporature[speed_index-1]

  speed_10=speed[speed_index]
  power_10=power[speed_index]

  power_10_small=[]
  power_10_large=[]

  for i in range(len(speed_index)):
      if temporature_list[i]<=np.mean(temporature_list):
          power_10_small.append(power[speed_index[i]])
      else:
          power_10_large.append(power[speed_index[i]])

  
  fig, axes = plt.subplots(figsize=(5, 3), dpi=300)

  sns.kdeplot(power_10_small, label=r'Small $a_{t-1}$', fill=True,)
  sns.kdeplot(power_10_large, label=r'Large $a_{t-1}$', fill=True,)
  axes.axvline(np.mean(power_10_small), label='Small mean', color='b', linewidth=2,  linestyle='--')
  axes.axvline(np.mean(power_10_large), label='Large mean', color='orange', linewidth=2, linestyle='--')
  axes.legend(prop={'size': 12},loc='best')
  #probplot(df[target], plot=axes[1])

  axes.tick_params(axis='x', labelsize=16)
  axes.tick_params(axis='y', labelsize=16)
  axes.set_xlabel('Standardized power',fontsize=16)
  axes.set_ylabel('Count',fontsize=16)
  axes.set_title('Speed Range:[%s,%s]'%(round(tmp_speed,1),round(tmp_speed+0.1,1)), fontsize=16, pad=12)
  plt.savefig('image_result/offline/temperature_%s.png'%round(tmp_speed,1))

Angle

In [ ]:
total_speed=np.arange(5,13,0.1)

speed=np.array(data_offline['speed(m/s)_%s'%item])
power=np.array(data_offline['power_%s'%item])
angle=np.array(data_offline['angle(°)_%s'%item])
power=(power-np.min(power))/(np.max(power)-np.min(power))

for tmp_speed in total_speed:

  speed_index=norm_data_offline[(norm_data_offline['speed(m/s)_%s'%item]>=tmp_speed)&(norm_data_offline['speed(m/s)_%s'%item]<=tmp_speed+0.1)].index
  
  angle_list=angle[speed_index-1]

  speed_10=speed[speed_index]
  power_10=power[speed_index]

  power_10_small=[]
  power_10_large=[]

  for i in range(len(speed_index)):
      if angle_list[i]<=np.mean(angle_list):
          power_10_small.append(power[speed_index[i]])
      else:
          power_10_large.append(power[speed_index[i]])

  
  fig, axes = plt.subplots(figsize=(5, 3), dpi=300)

  sns.kdeplot(power_10_small, label=r'Small $cos(\theta_{t-1})$', fill=True,)
  sns.kdeplot(power_10_large, label=r'Large $cos(\theta_{t-1})$', fill=True,)
  axes.axvline(np.mean(power_10_small), label='Small mean', color='b', linewidth=2,  linestyle='--')
  axes.axvline(np.mean(power_10_large), label='Large mean', color='orange', linewidth=2, linestyle='--')
  axes.legend(prop={'size': 12},loc='best')
  #probplot(df[target], plot=axes[1])

  axes.tick_params(axis='x', labelsize=16)
  axes.tick_params(axis='y', labelsize=16)
  axes.set_xlabel('Standardized power',fontsize=16)
  axes.set_ylabel('Count',fontsize=16)
  axes.set_title('Speed Range:[%s,%s]'%(round(tmp_speed,1),round(tmp_speed+0.1,1)), fontsize=16, pad=12)
  plt.savefig('image_result/offline/angle_%s.png'%round(tmp_speed,1))

Power

In [ ]:
total_speed=np.arange(5,13,0.1)

speed=np.array(data_offline['speed(m/s)_%s'%item])
power=np.array(data_offline['power_%s'%item])
angle=np.array(data_offline['angle(°)_%s'%item])
power=(power-np.min(power))/(np.max(power)-np.min(power))
power_2=np.array(data_offline['power(kW)_%s'%item])

for tmp_speed in total_speed:

  speed_index=norm_data_offline[(norm_data_offline['speed(m/s)_%s'%item]>=tmp_speed)&(norm_data_offline['speed(m/s)_%s'%item]<=tmp_speed+0.1)].index
  
  power_list=power_2[speed_index-1]

  speed_10=speed[speed_index]
  power_10=power[speed_index]

  power_10_small=[]
  power_10_large=[]

  for i in range(len(speed_index)):
      if power_list[i]<=np.mean(power_list):
          power_10_small.append(power[speed_index[i]])
      else:
          power_10_large.append(power[speed_index[i]])

  
  fig, axes = plt.subplots(figsize=(5, 3), dpi=300)

  sns.kdeplot(power_10_small, label=r'Small $p_{t-1}$', fill=True,)
  sns.kdeplot(power_10_large, label=r'Large $p_{t-1}$', fill=True,)
  axes.axvline(np.mean(power_10_small), label='Small mean', color='b', linewidth=2,  linestyle='--')
  axes.axvline(np.mean(power_10_large), label='Large mean', color='orange', linewidth=2, linestyle='--')
  axes.legend(prop={'size': 12},loc='best')
  #probplot(df[target], plot=axes[1])

  axes.tick_params(axis='x', labelsize=16)
  axes.tick_params(axis='y', labelsize=16)
  axes.set_xlabel('Wind Power(KW)',fontsize=16)
  axes.set_ylabel('Count',fontsize=16)
  axes.set_title('Speed Range:[%s,%s]'%(round(tmp_speed,1),round(tmp_speed+0.1,1)), fontsize=16, pad=12)
  plt.savefig('image_result/offline/power_%s.png'%round(tmp_speed,1))